In [8]:
import bitstring as bs

In [9]:
def next_float(f):
    #packing double-precision foat
    b = bs.pack('>d', f)
    
    #extracting mantisa as unsigned int and adding up 1.
    # There are two cases, (1) if the bits of the mantissa are all 1,
    # (2) all the other cases.
    m = b[12:].uint
    if m==4503599627370495:
        # Case (1)
        m=0
        b[12:] = m
        exp=b[1:12].uint
        exp +=1
        b[1:12] = exp
        
    else:
        # Case (2)
        m += 1
        # putting the result in his place
        b[12:] = m
    
    return b.float

def to_binary(f):
    b = bs.pack('>d', f)
    b = b.bin
    #show sign + exponent + mantisa
    print(b[0]+' '+b[1:12]+' '+b[12:])

In [159]:
def f_new_rep(x,bits_mant,bits_exp):
    # Algoritmo de representación de punto flotante modificada.
    n = bs.pack('>d', x) # Pasar el numero original a punto flotante
    b = n.bin

    # Signo
    signo = b[0]

    # Trabajar la mantisa
    mantisa = n[12:12+bits_mant]
    mantisa_b = n[12:12+bits_mant].uint
    if bits_mant < 52: # Si no se necesita la mantisa original completa
        if n[12 + bits_mant - 1] == '1' or int('0b' + b[bits_mant+12:], 2) > 2**(51-bits_mant):
            if mantisa_b == 2**bits_mant - 1:
                print('1')
                mantisa_b = 0
                n[12:12+bits_mant] = mantisa_b
                exp = n[1:12].uint
                exp += 1
                n[1:12] = exp
            else:
                print('2')
                mantisa_b += 1
                n[12:12+bits_mant] = mantisa_b
                print(b[12:12+bits_mant])
    mantisa = n[12:12+bits_mant].bin

    # Trabajar el exponente nuevo 
    exp = int('0b' + b[1:12], 2) - 1023 + ((2**(bits_exp-1))-1) # Calcular el valor decimal del nuevo exponente 
    exp = bin(exp)[2:] # Pasarlo a binario nuevamente
    exp = (bits_exp-len(exp))*'0' + exp # Agregarle 0's si hace falta

    result = signo + ' ' + exp + ' ' + mantisa  # Juntar todo

    return  result

In [160]:
# 1.2343
x = 3.3253
to_binary(x)
f_new_rep(x, 29, 7)

0 10000000000 1010100110100011011011100010111010110001110001000011
2
10101001101000110110111000101


'0 1000000 10101001101000110110111000110'

In [149]:
x = 10
x = bs.pack('>d', x)
x.bi

'0100000000100100000000000000000000000000000000000000000000000000'